<a href="https://colab.research.google.com/github/jbenasuli/final_project/blob/database/amazon_product_selection_outdoors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-le

In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [12]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

In [13]:
# Create a table to count
product_count = df.groupby("product_id").agg({"review_id":"count"}).withColumnRenamed("count(review_id)", "reviews_count")
product_count.show()

+----------+-------------+
|product_id|reviews_count|
+----------+-------------+
|B00C4ZBHPQ|            2|
|B000BR4NIC|           72|
|B000HBMFRS|         1797|
|B00KG3MRTS|           10|
|B010APZ8VK|            1|
|B000HJ7CLY|          512|
|B000FJR0P4|            2|
|B00JGHOA7M|           29|
|B007ZYIFQ0|          162|
|B00JMAB6MK|           18|
|B00G62JUJS|            1|
|B00FRSTN94|            5|
|B00629SK6O|            3|
|B00UTHCHRS|           64|
|B00JHHUAWA|           24|
|B00ESFQB0Q|           53|
|B00L9QPUMC|           82|
|B001BW69LU|            6|
|B00G7H790E|           81|
|B00ASSOHH6|           76|
+----------+-------------+
only showing top 20 rows



In [14]:
products_filtered = product_count.filter(product_count.reviews_count > 2000)
products_filtered.show()

+----------+-------------+
|product_id|reviews_count|
+----------+-------------+
|B000HBILB2|         3014|
|B006VPPERC|         2358|
|B000M0MJU2|         4753|
|B00FAW4O0A|         6033|
|B006QF3TW4|         3308|
|B006OU4ERA|         2292|
|B00A6TBITM|         2399|
|B00NPLSZF8|         2862|
+----------+-------------+

